# Classification d'incidents avec un réseau  récurrent unidirectionnel et des *embeddings* Spacy

## 1. Création du jeu de données (*dataset*)

In [24]:
import json
import spacy
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [25]:
train_json_fn = "./data/incidents_train.json"
validation_json_fn = "./data/incidents_test.json"
test_json_fn = "./data/incidents_test.json"

In [26]:
#Fonction permettant de charger les données
def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
    return incident_list

In [27]:
train_list = load_incident_dataset(train_json_fn)
validation_list = load_incident_dataset(validation_json_fn)
test_list = load_incident_dataset(test_json_fn)

print("Nombre d'incidents dans train:", len(train_list))
print("Nombre d'incidents dans validation:", len(validation_list))
print("Nombre d'incidents dans test:", len(test_list))

Nombre d'incidents dans train: 2475
Nombre d'incidents dans validation: 531
Nombre d'incidents dans test: 531


In [28]:
#On divise nos listes en X(texte) et y(labels) pour chacun des sets

X_train = [instance["text"] for instance in train_list]
y_train = [instance["label"] for instance in train_list]

X_val = [instance["text"] for instance in validation_list]
y_val = [instance["label"] for instance in validation_list]

X_test = [instance["text"] for instance in test_list]
y_test = [instance["label"] for instance in test_list]

In [29]:
nb_classes = len(set(y_train))

## 2. Gestion de plongements de mots (*embeddings*)

In [30]:
spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 542.5 kB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


On charge ici notre model spacy

In [31]:
nlp = spacy.load('en_core_web_md')
embedding_size = nlp.meta['vectors']['width']

Ce bloc de code définit des tokens spéciaux pour le remplissage PAD et les mots inconnus UNK, associant à ces tokens des identifiants (0 et 1) et des vecteurs d'embedding de taille zéro pour chacun, et crée des dictionnaires pour mapper les identifiants aux mots (id2word) et les mots aux identifiants (word2id), ainsi que les identifiants aux embeddings correspondants (id2embedding).

In [32]:
padding_token = "<PAD>"   # mot 0
unk_token = "<UNK>"    # mot 1
zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

id2word = {}
id2word[0] = padding_token 
id2word[1] = unk_token 

word2id = {}
word2id[padding_token] = 0
word2id[unk_token] = 1

id2embedding = {}
id2embedding[0] = zero_vec_embedding
id2embedding[1] = zero_vec_embedding

In [33]:
word_index = 2  # Initialise le compteur d'index à 2, car 0 et 1 sont réservés pour <PAD> et <UNK>
vocab = word2id.keys()  # Récupère les mots déjà existants dans le dictionnaire word2id

# Parcourt toutes les incidents dans l'ensemble d'entraînement X_train
for incident in X_train:
    # Tokenise chaque question en mots à l'aide de spacy
    for word in nlp(incident):
        # Vérifie si le mot n'est pas déjà dans le vocabulaire
        if word.text not in vocab:
            # Si le mot est nouveau, lui attribue le prochain index disponible
            word2id[word.text] = word_index
            # Ajoute le mot et son index correspondant au dictionnaire id2word
            id2word[word_index] = word.text
            # Stocke le vecteur d'embedding du mot dans le dictionnaire id2embedding
            id2embedding[word_index] = word.vector
            # Incrémente l'index pour le prochain mot unique
            word_index += 1

 La fonction ci-dessous sera utilisée par nos DataLoaders pour effectuer le rembourrage des mini-batchs.

In [34]:
def pad_batch(batch):
    # Extrait les entrées (x) et les longueurs réelles (x_true_length) de chaque séquence dans le batch
    x = [x for x, y in batch]  # Récupère les séquences d'entrée
    x_true_length = [len(x) for x, y in batch]  # Calcule la longueur de chaque séquence d'entrée

    # Extrait et empile les cibles (y) de chaque élément du batch en un tensor
    y = torch.stack([y for x, y in batch], dim=0)

    # Rembourre les séquences d'entrée pour qu'elles aient toutes la même longueur et les retourne avec leurs longueurs réelles
    return ((pad_sequence(x, batch_first=True), x_true_length), y)

## 3. Création de modèle(s)

Le réseau comprendra une couche d'embedding en entrée. Cette couche d'embedding sera utilisée pour générer les représentations vectorielles (embeddings) de chaque mot dans une phrase. Les inputs fournis à notre réseau seront des batchs contenant des listes de phrases, où chaque mot est encodé par son identifiant (ID) correspondant. Ces listes seront rembourrées (padded) pour assurer une longueur uniforme au sein d'un batch.

En plus des données de la phrase, nous fournirons également à notre fonction une variable x_length. Cette variable est importante pour la fonction pack_padded_sequence de PyTorch, qui permet de gérer efficacement les phrases de longueurs variables dans un batch. pack_padded_sequence crée une représentation compacte de ces batchs, en ignorant les éléments de padding, ce qui améliore l'efficacité du traitement par le réseau.

In [35]:
class LSTMClassifier(nn.Module):
    def __init__(self, embeddings, hidden_state_size, nb_classes) :
        super(LSTMClassifier, self).__init__()
        # Initialise une couche d'embedding à partir d'embeddings pré-entraînés
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        # Détermine la taille des embeddings
        self.embedding_size = embeddings.size()[1] 
        # Initialise un LSTM avec une seule couche, unidirectionnel  
        self.rnn = nn.LSTM(self.embedding_size, hidden_state_size, 1, batch_first=True)
        self.classification_layer = nn.Linear(hidden_state_size, nb_classes)
    
    def forward(self, x, x_lengths):
        # Passe les données d'entrée par la couche d'embedding
        x = self.embedding_layer(x)
        # Emballe les séquences rembourrées pour le traitement par LSTM
        packed_batch = pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
        output, (h_n, c_n) = self.rnn(packed_batch)  # On utilise le hidden state de la dernière cellule
        x = h_n.squeeze()  # Le LSTM a une seule couche, on retire cette dimension
        x = self.classification_layer(x)
        return x

Dans le bloc de code ci-dessous, une matrice d'embeddings est créée et remplie avec les vecteurs d'embeddings pour chaque mot du vocabulaire, en utilisant un dictionnaire qui mappe les identifiants de mots à leurs embeddings correspondants. Cette matrice est ensuite convertie en un tensor PyTorch, permettant son utilisation dans des réseaux de neurones, et la taille de la couche d'embeddings créée est affichée pour vérification.

In [36]:
# Calcule la taille du vocabulaire basé sur le dictionnaire id2embedding
vocab_size = len(id2embedding)
# Crée une matrice d'embeddings initialisée à zéro avec la taille du vocabulaire et la taille des embeddings
embedding_layer = np.zeros((vocab_size, embedding_size), dtype=np.float32)

# Remplit la matrice d'embeddings avec les embeddings correspondants pour chaque ID de token
for token_id, embedding in id2embedding.items():
    embedding_layer[token_id, :] = embedding  # Affecte l'embedding à la ligne correspondant à l'ID du token

# Convertit la matrice d'embeddings de numpy à un tensor PyTorch
embedding_layer = torch.from_numpy(embedding_layer)

print("Taille de la couche d'embeddings:", embedding_layer.shape)

Taille de la couche d'embeddings: torch.Size([11642, 300])


## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

Ici, nous définissons une classe Dataset qui permet de construire notre dataset à partir de listes de phrases et de leurs classes correspondantes, en utilisant le dictionnaire word2id défini précédemment. Ce dataset sera ensuite fourni à un DataLoader, qui se chargera de diviser le dataset en batches et d'effectuer le rembourrage nécessaire avec la fonction pad_batch, afin que toutes nos entrées aient la même taille.

In [37]:
class SpacyDataset(Dataset):
    def __init__(self, data , targets, word_to_id, spacy_model):
        self.data = data
        self.sequences = [None for _ in range(len(data))]
        self.targets = targets
        self.word2id = word_to_id
        self.tokenizer = spacy_model
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Vérifie si la séquence à l'index spécifié a déjà été tokenisée
        if self.sequences[index] is None:
            # Si non, tokenize la phrase à cet index et stocke le résultat dans self.sequences
            self.sequences[index] = self.tokenize(self.data[index]) 
        return LongTensor(self.sequences[index]), LongTensor([int(self.targets[index])]).squeeze(0)


    def tokenize(self, sentence):
        # Utilise le modèle spaCy pour tokeniser la phrase donnée
        tokens = [word.text for word in self.tokenizer(sentence)]
        # Convertit chaque token en son identifiant correspondant.
        # Si le token n'est pas trouvé dans dictionnaire word2id, utilise 1 par défaut, qui est l'identifiant pour <UNK> (mot inconnu).
        return [self.word2id.get(token, 1) for token in tokens]

## 5. Entraînement de modèle(s)

Pour l'entraînement du modèle, nous utiliserons la bibliothèque Poutyne, qui permet d'automatiser ce processus.

In [38]:
train_dataset = SpacyDataset(X_train, y_train, word2id, nlp)
valid_dataset = SpacyDataset(X_val, y_val, word2id, nlp)
test_dataset = SpacyDataset(X_test, y_test, word2id, nlp)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)

Le choix d'une taille de couche cachée de 150 pour le RNN est principalement justifié par la performance empirique observée lors des tests. Après avoir expérimenté avec différentes tailles de couches cachées, il a été constaté que 150 unités offraient un équilibre optimal en termes de précision, de perte de test, et de temps de calcul.

In [39]:
hidden_dimension = 150

In [40]:
directory_name = 'model_t2/_rnnUni'

model = LSTMClassifier(embedding_layer, hidden_dimension, nb_classes)
experiment = Experiment(directory_name, 
                                model, 
                                optimizer = "Adam", 
                                task="classification")
        
logging = experiment.train(train_dataloader, valid_dataloader, epochs=10, disable_tensorboard=True)

Epoch:  1/10 Train steps: 155 Val steps: 34 2m36.72s loss: 1.620720 acc: 44.080808 fscore_macro: 0.140311 val_loss: 1.383634 val_acc: 50.659134 val_fscore_macro: 0.197170
Epoch 1: val_acc improved from -inf to 50.65913, saving file to model_t2/_rnnUni/checkpoint_epoch_1.ckpt
Epoch:  2/10 Train steps: 155 Val steps: 34 40.97s loss: 1.151229 acc: 60.363636 fscore_macro: 0.304882 val_loss: 1.120377 val_acc: 60.828625 val_fscore_macro: 0.337122
Epoch 2: val_acc improved from 50.65913 to 60.82863, saving file to model_t2/_rnnUni/checkpoint_epoch_2.ckpt
Epoch:  3/10 Train steps: 155 Val steps: 34 40.38s loss: 0.899050 acc: 68.525253 fscore_macro: 0.423974 val_loss: 1.029107 val_acc: 63.465160 val_fscore_macro: 0.419636
Epoch 3: val_acc improved from 60.82863 to 63.46516, saving file to model_t2/_rnnUni/checkpoint_epoch_3.ckpt
Epoch:  4/10 Train steps: 155 Val steps: 34 43.04s loss: 0.743714 acc: 74.303030 fscore_macro: 0.527109 val_loss: 1.019130 val_acc: 65.536723 val_fscore_macro: 0.485139

## 6. Évaluation et analyse de résultats

In [41]:
experiment.test(test_dataloader)

Found best checkpoint at epoch: 5
lr: 0.001, loss: 0.607008, acc: 78.7879, fscore_macro: 0.631114, val_loss: 1.00264, val_acc: 67.42, val_fscore_macro: 0.587087
Loading checkpoint model_t2/_rnnUni/checkpoint_epoch_5.ckpt
Running test
Test steps: 34 11.71s test_loss: 1.002641 test_acc: 67.419962 test_fscore_macro: 0.587087          


{'time': 11.710461083000155,
 'test_loss': 1.002640670414474,
 'test_acc': 67.41996236395252,
 'test_fscore_macro': 0.5870870351791382}

## Analyse des resultats ##

Une exactitude de 67.42% indique que le modèle est capable de prédire correctement environ deux tiers des cas de test, ce qui est une performance respectable, mais qui pourrait être améliorée pour certaines applications exigeant une précision plus élevée.